# getting data

downloading data from kaggle

In [15]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "drive/MyDrive/kaggle/"

In [48]:
!kaggle competitions download -c data-science-bowl-2018

 94% 259M/276M [00:01<00:00, 207MB/s]
100% 276M/276M [00:01<00:00, 203MB/s]
  0% 0.00/112k [00:00<?, ?B/s]
100% 112k/112k [00:00<00:00, 93.7MB/s]
 78% 62.0M/79.1M [00:00<00:00, 209MB/s]
100% 79.1M/79.1M [00:00<00:00, 199MB/s]
  0% 0.00/386k [00:00<?, ?B/s]
100% 386k/386k [00:00<00:00, 115MB/s]
  0% 0.00/9.10M [00:00<?, ?B/s]
100% 9.10M/9.10M [00:00<00:00, 149MB/s]
  0% 0.00/2.62k [00:00<?, ?B/s]
100% 2.62k/2.62k [00:00<00:00, 2.51MB/s]
  0% 0.00/2.67M [00:00<?, ?B/s]
100% 2.67M/2.67M [00:00<00:00, 180MB/s]


extracting training data

In [33]:
import zipfile
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import shutil

from skimage.io import *
from skimage.transform import resize

In [49]:
with zipfile.ZipFile("stage1_train.zip", "r") as file:
  file.extractall("train_data/")

In [50]:
TRAIN_DIR = "train_data/"

for fname in tqdm(os.listdir(TRAIN_DIR)):
    m_path = TRAIN_DIR + str(fname) + "/masks/"
    s_path = TRAIN_DIR + str(fname) + "/mask/"
    
    mask = np.zeros((256, 256, 1), dtype=np.uint8)
    
    for mask_f in next(os.walk(m_path))[2]:
        mask_ = imread(m_path+ mask_f)
        mask_ = np.expand_dims(resize(mask_, (256,256), mode='constant', preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    
    os.mkdir(s_path)
    imsave(s_path+fname+".jpg", mask)
    shutil.rmtree(m_path)

  2%|▏         | 12/670 [00:06<04:12,  2.61it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: train_data/e5aeb5b3577abbebe8982b5dd7d22c4257250ad3000661a42f38bf9248d291fd/mask/e5aeb5b3577abbebe8982b5dd7d22c4257250ad3000661a42f38bf9248d291fd.jpg is a low contrast image
  from ipykernel import kernelapp as app
 10%|█         | 69/670 [00:33<04:46,  2.10it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: train_data/af576e8ec3a8d0b57eb6a311299e9e4fd2047970d3dd9d6f52e54ea6a91109da/mask/af576e8ec3a8d0b57eb6a311299e9e4fd2047970d3dd9d6f52e54ea6a91109da.jpg is a low contrast image
  from ipykernel import kernelapp as app
 11%|█         | 71/670 [00:33<03:31,  2.84it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: train_data/58c593bcb98386e7fd42a1d34e291db93477624b164e83ab2afa3caa90d1d921/mask/58c593bcb98386e7fd42a1d34e291db93477624b164e83ab2afa3caa90d1d921.jpg is a low contrast image
  from ipykernel i